In [1]:
import numpy as np
import cv2
import tkinter as tk
from tkinter import filedialog

In [2]:
piel = []
npiel = []
def on_mouse_click(event, x, y, flags, param):
    global skin_samples, bg_samples
    if event == cv2.EVENT_LBUTTONDOWN:  # Clic izquierdo
        pixel = image[y, x]
        print(f"Ubicación del pixel: ({x}, {y}) coordenadas RGB: {pixel}")
        piel.append(list(pixel))
    elif event == cv2.EVENT_RBUTTONDOWN:  # Clic derecho
        pixel = image[y, x]
        print(f"Ubicación del pixel: ({x}, {y}) coordenadas RGB: {pixel}")
        npiel.append(list(pixel))

In [3]:
root = tk.Tk()
root.withdraw() 
file_path = 'piel.jpg'

image = cv2.imread(file_path)

cv2.namedWindow('Image')
cv2.setMouseCallback('Image', on_mouse_click)

while True:
    cv2.imshow('Image', image)
    key = cv2.waitKey(1) & 0xFF
    if key == 27:  # Presionar Esc para salir
        break

Ubicación del pixel: (415, 508) coordenadas RGB: [25 21 26]
Ubicación del pixel: (345, 545) coordenadas RGB: [27 26 28]
Ubicación del pixel: (229, 563) coordenadas RGB: [31 31 37]
Ubicación del pixel: (142, 550) coordenadas RGB: [30 29 33]
Ubicación del pixel: (117, 491) coordenadas RGB: [19 18 27]
Ubicación del pixel: (76, 493) coordenadas RGB: [18 17 19]
Ubicación del pixel: (95, 568) coordenadas RGB: [28 27 29]
Ubicación del pixel: (95, 382) coordenadas RGB: [255 255 255]
Ubicación del pixel: (452, 359) coordenadas RGB: [255 255 255]
Ubicación del pixel: (261, 99) coordenadas RGB: [19 30 44]
Ubicación del pixel: (243, 174) coordenadas RGB: [167 184 223]
Ubicación del pixel: (198, 181) coordenadas RGB: [166 184 225]
Ubicación del pixel: (269, 283) coordenadas RGB: [189 196 235]
Ubicación del pixel: (188, 283) coordenadas RGB: [185 190 235]
Ubicación del pixel: (224, 359) coordenadas RGB: [165 184 221]
Ubicación del pixel: (308, 337) coordenadas RGB: [158 177 212]
Ubicación del pixel:

In [6]:
print(len(piel))
print(len(npiel))

10
10


In [8]:
skin_mean = np.mean(piel, axis=0)
bg_mean = np.mean(npiel, axis=0)
skin_cov = np.cov(np.array(piel).T)
bg_cov = np.cov(np.array(npiel).T)

In [9]:
def gaussian_prob(pixel, mean, cov):
    pixel = np.array(pixel)
    return 1 / (np.sqrt((2 * np.pi) ** 3 * np.linalg.det(cov))) * \
           np.exp(-0.5 * np.dot(np.dot((pixel - mean).T, np.linalg.inv(cov)), (pixel - mean)))

In [10]:
def classify_pixel(pixel, skin_gaussian, bg_gaussian, bg_prior):
    p_skin = gaussian_prob(pixel, skin_gaussian['mean'], skin_gaussian['cov'])
    p_bg = gaussian_prob(pixel, bg_gaussian['mean'], bg_gaussian['cov'])
    posterior_skin = (p_skin * bg_prior) / (p_skin * bg_prior + p_bg * (1 - bg_prior))
    return posterior_skin > 0.5

In [12]:
bg_prior = len(npiel) / (len(piel) + len(npiel))

skin_gaussian = {'mean': skin_mean, 'cov': skin_cov}
bg_gaussian = {'mean': bg_mean, 'cov': bg_cov}

In [13]:
segmented_image = np.zeros_like(image)
for i in range(image.shape[0]):
    for j in range(image.shape[1]):
        pixel = image[i, j]
        is_skin = classify_pixel(pixel, skin_gaussian, bg_gaussian, bg_prior)
        if is_skin:
            segmented_image[i, j] = [255, 255, 255]  
        else:
            segmented_image[i, j] = [0, 0, 0]   

In [14]:
# Mostrar la imagen segmentada
cv2.imshow('Segmented Image', segmented_image)

# Guardar la imagen segmentada en el disco
output_file = "segmented_image_casa_clics.png"  # Nombre del archivo de salida
cv2.imwrite(output_file, segmented_image)
print(f"Imagen segmentada guardada como {output_file}")

cv2.waitKey(0)
cv2.destroyAllWindows()

Imagen segmentada guardada como segmented_image_casa_clics.png
